In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from scipy.spatial import distance_matrix
import scipy as sp
import random
import time

# Custom modules
import funcs
import diff_map_funcs
import CC_weights_graphs
import FBS_algo    # Contains fbs_gme3_cc_mat (the baseline implementation)
import admm_cc

from tqdm.notebook import tqdm

In [2]:
def run_one_trial_fbs_gme3_cc(
    fbs_func,
    p=100,
    K=4,
    n_in_cluster=5,
    random_seed=1,
    gamma=None,
    print_graph='y'
):
    """
    Runs a single experiment:
      1) Generate Gaussian clusters
      2) Create MST + KNN graph
      3) Run GME3-CC FBS solver (provided via fbs_func)
      4) Perform cluster fusion
      5) Return CPU time and residual
    
    Parameters
    ----------
    fbs_func: function
        A function pointer of signature:
          fbs_func(D, C, sig1_C, gamma, U0, data)
        e.g., FBS_algo.fbs_gme3_cc_mat
    p: int
        Dimension of the data (assumes p > 2 for projection).
    K: int
        Number of Gaussian clusters.
    n_in_cluster: int
        Number of data points per cluster.
    random_seed: int
        Random seed to generate the data (for reproducibility).
    gamma: float or None
        Regularization parameter for GME3-CC. If None, auto-set within function.
    print_graph: str
        If 'y', prints MST steps from CC_weights_graphs (for debugging).
    
    Returns
    -------
    cput : float
        CPU time (in seconds) for the FBS solver.
    re_tol : float
        Final residual (tolerance) from FBS.
    U_opt : np.ndarray
        The final fused matrix U of shape (p, n).
    """

    #-----------------------------
    # 1) Generate data
    #-----------------------------
    n = K * n_in_cluster
    GM_data, labels, means, covariances = funcs.generate_gaussian_clusters(
        K, p, n_in_cluster, random_seed=random_seed
    )
    matrixData = GM_data.T
    
    #-----------------------------
    # 2) Create MST + KNN graph
    #-----------------------------
    k_nrst = n_in_cluster - 1
    weight_mat0 = CC_weights_graphs.assign_weights(matrixData, k_nrst)
    D, weights_vec = CC_weights_graphs.create_graph_MST_KNN(
        matrixData, weight_mat0, k_nrst, print_G=print_graph
    )
    
    #-----------------------------
    # Prepare for FBS
    #-----------------------------
    # Identity matrices
    I_n = sp.sparse.eye(n).toarray()
    C = sp.sparse.eye(D.shape[0]).toarray() @ np.diag(weights_vec)
    B = C @ D
    sig1_B = np.linalg.norm(B, 2)
    sig1_C = np.linalg.norm(C, 2)
    
    # If gamma is not supplied, pick a default in [1e-6, 1/(sig1_B^2) + 5]
    if gamma is None:
        gamma = 0.5 * (1/(sig1_B**2) + 5)
    
    # Initialize U = the data
    U0 = matrixData.copy()
    
    #-----------------------------
    # 3) Run FBS solver
    #-----------------------------
    start_time = time.time()
    U_opt, V_opt, Z_opt, cput, re_tol = fbs_func(D, C, sig1_C, gamma, U0, matrixData)
    elapsed_time = time.time() - start_time

    #-----------------------------
    # 4) Perform cluster fusion
    #-----------------------------
    # We fuse columns of U_opt that are "close" (based on D*U).
    tol_sim_path = 1e-3
    DU = C @ D @ U_opt.T  
    differences = np.linalg.norm(DU.T, axis=0)
    connected_ix = np.where(differences <= tol_sim_path)[0]
    
    AA = sp.sparse.lil_matrix((n, n), dtype=int)
    for kk in connected_ix:
        ii, jj = np.where(D[kk, :] != 0)[0]
        AA[ii, jj] = 1
        AA[jj, ii] = 1

    # find_clusters returns a dictionary with 'cluster' and 'size'
    rrr = funcs.find_clusters(AA)
    uni_clusters = rrr['cluster']
    uni_clusters_id = np.unique(uni_clusters)
    
    U_sim = U_opt.copy()
    for cluster_id in uni_clusters_id:
        cluster_idx = np.where(uni_clusters == cluster_id)[0]
        col_mean = np.mean(U_opt[:, cluster_idx], axis=1)
        U_sim[:, cluster_idx] = np.tile(col_mean, (len(cluster_idx), 1)).T
    
    return cput, re_tol, U_opt

In [3]:
def run_multiple_trials_fbs_gme3_cc(
    fbs_func,
    n_trials=5,
    p=100,
    K=4,
    n_in_cluster=5,
    seed_list=None,
    gamma=None,
    print_graph='n'
):
    """
    Runs multiple trials of the GME3-CC FBS experiment with different random seeds.
    
    Parameters
    ----------
    fbs_func : function
        Implementation of fbs_gme3_cc_mat to test.
    n_trials : int
        Number of trials to run.
    p : int
        Dimension of data (p > 2).
    K : int
        Number of clusters.
    n_in_cluster : int
        Number of points per cluster.
    seed_list : list or None
        List of random seeds. If None, [1, 2, ..., n_trials].
    gamma : float or None
        Regularization param. If None, automatically computed in each trial.
    print_graph : str
        If 'y', prints MST steps from CC_weights_graphs (for debugging).
    
    Returns
    -------
    cput_list : list of float
        List of CPU times for each trial.
    re_tol_list : list of float
        List of final residuals for each trial.
    """

    if seed_list is None:
        seed_list = list(range(1, n_trials + 1))
    
    cput_list = []
    re_tol_list = []
    
    for i, seed in enumerate(tqdm(seed_list)):
        cput, re_tol, U_opt = run_one_trial_fbs_gme3_cc(
            fbs_func=fbs_func,
            p=p,
            K=K,
            n_in_cluster=n_in_cluster,
            random_seed=seed,
            gamma=gamma,
            print_graph=print_graph
        )
        cput_list.append(cput)
        re_tol_list.append(re_tol)
        print(f"Trial {i+1}/{n_trials}, Seed={seed}, CPU={cput:.4f}, Residual={re_tol:.4e}")
    
    return cput_list, re_tol_list

In [4]:
"""trial parameters """
n_trails = 10
p = 10
K = 4
n_in_cluster = 20

In [5]:
cput_list, re_tol_list = run_multiple_trials_fbs_gme3_cc(
    fbs_func=FBS_algo.fbs_gme3_cc_mat,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

MPD Converged in 15 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 14 iterations.
FBS reaches max step 10.
FBS res: 0.000780442844952994
Trial 3/10, Seed=3, CPU=32.3273, Residual=7.8044e-04
Edge (0, 8) has different weights in D1 and D2: 0.9862799434950493 vs 0.9862799434950492
Edge (3, 16) has different weights in D1 and D2: 0.97764721188625 vs 0.9776472118862499
Edge (4, 8) has different weights in D1 and D2: 0.9909056135899924 vs 0.99

MPD Converged in 6 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 11 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 2 iterations.
ADMM Converged in 13 iterations.
FBS reaches max step 10.
FBS res: 0.0009320299909800723
Trial 5/10, Seed=5, CPU=29.3749, Residual=9.3203e-04
Edge (6, 12) has different weights in D1 and D2: 0.9846025675509958 vs 0.9846025675509957
Edge (8, 15) has different weights in D1 and D2: 0.9955357371005883 vs 0.9955357371005882
Edge (11, 13) has different weights in D1 and D2: 0.9853502425484794 v

Edge (1, 11) has different weights in D1 and D2: 0.9822897455867642 vs 0.9822897455867643
Edge (1, 9) has different weights in D1 and D2: 0.9775832193930524 vs 0.9775832193930523
Edge (2, 18) has different weights in D1 and D2: 0.9685218640029833 vs 0.9685218640029835
Edge (7, 13) has different weights in D1 and D2: 0.985025588808146 vs 0.9850255888081458
Edge (7, 16) has different weights in D1 and D2: 0.9778924058780931 vs 0.9778924058780932
Edge (9, 14) has different weights in D1 and D2: 0.9838853771578108 vs 0.9838853771578109
Edge (11, 15) has different weights in D1 and D2: 0.9820699314179888 vs 0.9820699314179889
Edge (12, 13) has different weights in D1 and D2: 0.9713810695277258 vs 0.9713810695277257
Edge (20, 37) has different weights in D1 and D2: 0.9618021616649084 vs 0.9618021616649083
Edge (21, 27) has different weights in D1 and D2: 0.9747986581494517 vs 0.9747986581494518
Edge (21, 25) has different weights in D1 and D2: 0.9585417444030084 vs 0.9585417444030085
Edge (2

Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edge (20, 36) has different weights in D1 and D2: 0.9785802580925324 vs 0.9785802580925322
Edge (20, 33) has different weights in D1 and D2: 0.9653963685045024 vs 0.9653963685045023
Edge (23, 27) has different weights in D1 and D2: 0.9651199900995031 vs 0.9651199900995032
Edge (25, 30) has different weights in D1 and D2: 0.9726364584496663 vs 0.9726364584496662
Edge (2

In [6]:
"""Original implementation """
print("CPU times:", cput_list)
print("Residuals:", re_tol_list)
print(f"Avg CPU time = {np.mean(cput_list):.4f} ({np.std(cput_list):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list):.4f} ({np.std(re_tol_list):.4f}) seconds")

CPU times: [30.72841191291809, 29.61775803565979, 32.327343225479126, 28.8537175655365, 29.374895572662354, 27.455753564834595, 29.6502845287323, 29.554741382598877, 30.64226460456848, 30.082072734832764]
Residuals: [0.0006822281057085706, 0.0014770883398889094, 0.000780442844952994, 0.0007087772629708857, 0.0009320299909800723, 0.0009361300007083585, 0.0007263101357284762, 0.0011257631513639575, 0.001802063256680626, 0.0008262251568901267]
Avg CPU time = 29.8287 (1.2132) seconds
Avg residual = 0.0010 (0.0004) seconds


In [7]:
fbs_gme3_cc_mat_partial = lambda D, C, sig1_C, gamma, U0, mat_x : FBS_algo.fbs_gme3_cc_mat(D, C, sig1_C, gamma, U0, mat_x)
cput_list1, re_tol_list1 = run_multiple_trials_fbs_gme3_cc(
    fbs_func=fbs_gme3_cc_mat_partial,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

MPD Converged in 15 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 14 iterations.
FBS reaches max step 10.
FBS res: 0.000780442844952994
Trial 3/10, Seed=3, CPU=30.9365, Residual=7.8044e-04
Edge (0, 8) has different weights in D1 and D2: 0.9862799434950493 vs 0.9862799434950492
Edge (3, 16) has different weights in D1 and D2: 0.97764721188625 vs 0.9776472118862499
Edge (4, 8) has different weights in D1 and D2: 0.9909056135899924 vs 0.99

MPD Converged in 6 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 11 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 2 iterations.
ADMM Converged in 13 iterations.
FBS reaches max step 10.
FBS res: 0.0009320299909800723
Trial 5/10, Seed=5, CPU=28.3324, Residual=9.3203e-04
Edge (6, 12) has different weights in D1 and D2: 0.9846025675509958 vs 0.9846025675509957
Edge (8, 15) has different weights in D1 and D2: 0.9955357371005883 vs 0.9955357371005882
Edge (11, 13) has different weights in D1 and D2: 0.9853502425484794 v

Edge (1, 11) has different weights in D1 and D2: 0.9822897455867642 vs 0.9822897455867643
Edge (1, 9) has different weights in D1 and D2: 0.9775832193930524 vs 0.9775832193930523
Edge (2, 18) has different weights in D1 and D2: 0.9685218640029833 vs 0.9685218640029835
Edge (7, 13) has different weights in D1 and D2: 0.985025588808146 vs 0.9850255888081458
Edge (7, 16) has different weights in D1 and D2: 0.9778924058780931 vs 0.9778924058780932
Edge (9, 14) has different weights in D1 and D2: 0.9838853771578108 vs 0.9838853771578109
Edge (11, 15) has different weights in D1 and D2: 0.9820699314179888 vs 0.9820699314179889
Edge (12, 13) has different weights in D1 and D2: 0.9713810695277258 vs 0.9713810695277257
Edge (20, 37) has different weights in D1 and D2: 0.9618021616649084 vs 0.9618021616649083
Edge (21, 27) has different weights in D1 and D2: 0.9747986581494517 vs 0.9747986581494518
Edge (21, 25) has different weights in D1 and D2: 0.9585417444030084 vs 0.9585417444030085
Edge (2

Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edge (20, 36) has different weights in D1 and D2: 0.9785802580925324 vs 0.9785802580925322
Edge (20, 33) has different weights in D1 and D2: 0.9653963685045024 vs 0.9653963685045023
Edge (23, 27) has different weights in D1 and D2: 0.9651199900995031 vs 0.9651199900995032
Edge (25, 30) has different weights in D1 and D2: 0.9726364584496663 vs 0.9726364584496662
Edge (2

In [8]:
"""Lambda implementation """
print("CPU times:", cput_list1)
print("Residuals:", re_tol_list1)
print(f"Avg CPU time = {np.mean(cput_list1):.4f} ({np.std(cput_list1):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list1):.4f} ({np.std(re_tol_list1):.4f}) seconds")

CPU times: [29.130462646484375, 28.262853860855103, 30.93647027015686, 28.234016180038452, 28.332445859909058, 28.378201484680176, 29.181379079818726, 28.67852282524109, 28.950276613235474, 27.237037181854248]
Residuals: [0.0006822281057085706, 0.0014770883398889094, 0.000780442844952994, 0.0007087772629708857, 0.0009320299909800723, 0.0009361300007083585, 0.0007263101357284762, 0.0011257631513639575, 0.001802063256680626, 0.0008262251568901267]
Avg CPU time = 28.7322 (0.9088) seconds
Avg residual = 0.0010 (0.0004) seconds


In [11]:
fbs_gme3_cc_mat_partial = lambda D, C, sig1_C, gamma, U0, mat_x : FBS_algo.fbs_gme3_cc_mat_proto(D, C, sig1_C, gamma, U0, mat_x, admm=FBS_algo.prox_mat_l21_admm_vectorizedl21)
cput_list1, re_tol_list1 = run_multiple_trials_fbs_gme3_cc(
    fbs_func=fbs_gme3_cc_mat_partial,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

MPD Converged in 15 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 14 iterations.
FBS reaches max step 10.
FBS res: 0.000780442844952994
Trial 3/10, Seed=3, CPU=5.8467, Residual=7.8044e-04
Edge (0, 8) has different weights in D1 and D2: 0.9862799434950493 vs 0.9862799434950492
Edge (3, 16) has different weights in D1 and D2: 0.97764721188625 vs 0.9776472118862499
Edge (4, 8) has different weights in D1 and D2: 0.9909056135899924 vs 0.990

MPD Converged in 6 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 11 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 2 iterations.
ADMM Converged in 13 iterations.
FBS reaches max step 10.
FBS res: 0.0009320299909800723
Trial 5/10, Seed=5, CPU=4.3017, Residual=9.3203e-04
Edge (6, 12) has different weights in D1 and D2: 0.9846025675509958 vs 0.9846025675509957
Edge (8, 15) has different weights in D1 and D2: 0.9955357371005883 vs 0.9955357371005882
Edge (11, 13) has different weights in D1 and D2: 0.9853502425484794 vs

Edge (1, 11) has different weights in D1 and D2: 0.9822897455867642 vs 0.9822897455867643
Edge (1, 9) has different weights in D1 and D2: 0.9775832193930524 vs 0.9775832193930523
Edge (2, 18) has different weights in D1 and D2: 0.9685218640029833 vs 0.9685218640029835
Edge (7, 13) has different weights in D1 and D2: 0.985025588808146 vs 0.9850255888081458
Edge (7, 16) has different weights in D1 and D2: 0.9778924058780931 vs 0.9778924058780932
Edge (9, 14) has different weights in D1 and D2: 0.9838853771578108 vs 0.9838853771578109
Edge (11, 15) has different weights in D1 and D2: 0.9820699314179888 vs 0.9820699314179889
Edge (12, 13) has different weights in D1 and D2: 0.9713810695277258 vs 0.9713810695277257
Edge (20, 37) has different weights in D1 and D2: 0.9618021616649084 vs 0.9618021616649083
Edge (21, 27) has different weights in D1 and D2: 0.9747986581494517 vs 0.9747986581494518
Edge (21, 25) has different weights in D1 and D2: 0.9585417444030084 vs 0.9585417444030085
Edge (2

Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edge (20, 36) has different weights in D1 and D2: 0.9785802580925324 vs 0.9785802580925322
Edge (20, 33) has different weights in D1 and D2: 0.9653963685045024 vs 0.9653963685045023
Edge (23, 27) has different weights in D1 and D2: 0.9651199900995031 vs 0.9651199900995032
Edge (25, 30) has different weights in D1 and D2: 0.9726364584496663 vs 0.9726364584496662
Edge (2

In [12]:
"""Vectorized-L21 implementation """

print("CPU times:", cput_list1)
print("Residuals:", re_tol_list1)
print(f"Avg CPU time = {np.mean(cput_list1):.4f} ({np.std(cput_list1):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list1):.4f} ({np.std(re_tol_list1):.4f}) seconds")

CPU times: [27.081997632980347, 11.716490745544434, 5.846736431121826, 8.105542659759521, 4.301715135574341, 6.013634443283081, 30.38016366958618, 28.992157697677612, 31.06398844718933, 30.24851894378662]
Residuals: [0.0006822281057085706, 0.0014770883398889094, 0.000780442844952994, 0.0007087772629708857, 0.0009320299909800723, 0.0009361300007083585, 0.0007263101357284762, 0.0011257631513639575, 0.001802063256680626, 0.0008262251568901267]
Avg CPU time = 18.3751 (11.3678) seconds
Avg residual = 0.0010 (0.0004) seconds


In [9]:
fbs_gme3_cc_mat_partial = lambda D, C, sig1_C, gamma, U0, mat_x : FBS_algo.fbs_gme3_cc_mat_proto(D, C, sig1_C, gamma, U0, mat_x, admm=FBS_algo.prox_mat_l21_admm_preinv)
cput_list1, re_tol_list1 = run_multiple_trials_fbs_gme3_cc(
    fbs_func=fbs_gme3_cc_mat_partial,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

MPD Converged in 15 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 14 iterations.
FBS reaches max step 10.
FBS res: 0.0007804428449526912
Trial 3/10, Seed=3, CPU=4.1176, Residual=7.8044e-04
Edge (0, 8) has different weights in D1 and D2: 0.9862799434950493 vs 0.9862799434950492
Edge (3, 16) has different weights in D1 and D2: 0.97764721188625 vs 0.9776472118862499
Edge (4, 8) has different weights in D1 and D2: 0.9909056135899924 vs 0.99

MPD Converged in 6 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 11 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 2 iterations.
ADMM Converged in 13 iterations.
FBS reaches max step 10.
FBS res: 0.0009320299909800737
Trial 5/10, Seed=5, CPU=4.0858, Residual=9.3203e-04
Edge (6, 12) has different weights in D1 and D2: 0.9846025675509958 vs 0.9846025675509957
Edge (8, 15) has different weights in D1 and D2: 0.9955357371005883 vs 0.9955357371005882
Edge (11, 13) has different weights in D1 and D2: 0.9853502425484794 vs

Edge (1, 11) has different weights in D1 and D2: 0.9822897455867642 vs 0.9822897455867643
Edge (1, 9) has different weights in D1 and D2: 0.9775832193930524 vs 0.9775832193930523
Edge (2, 18) has different weights in D1 and D2: 0.9685218640029833 vs 0.9685218640029835
Edge (7, 13) has different weights in D1 and D2: 0.985025588808146 vs 0.9850255888081458
Edge (7, 16) has different weights in D1 and D2: 0.9778924058780931 vs 0.9778924058780932
Edge (9, 14) has different weights in D1 and D2: 0.9838853771578108 vs 0.9838853771578109
Edge (11, 15) has different weights in D1 and D2: 0.9820699314179888 vs 0.9820699314179889
Edge (12, 13) has different weights in D1 and D2: 0.9713810695277258 vs 0.9713810695277257
Edge (20, 37) has different weights in D1 and D2: 0.9618021616649084 vs 0.9618021616649083
Edge (21, 27) has different weights in D1 and D2: 0.9747986581494517 vs 0.9747986581494518
Edge (21, 25) has different weights in D1 and D2: 0.9585417444030084 vs 0.9585417444030085
Edge (2

Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edge (20, 36) has different weights in D1 and D2: 0.9785802580925324 vs 0.9785802580925322
Edge (20, 33) has different weights in D1 and D2: 0.9653963685045024 vs 0.9653963685045023
Edge (23, 27) has different weights in D1 and D2: 0.9651199900995031 vs 0.9651199900995032
Edge (25, 30) has different weights in D1 and D2: 0.9726364584496663 vs 0.9726364584496662
Edge (2

In [10]:
"""Pre-inverse implementation """
print("CPU times:", cput_list1)
print("Residuals:", re_tol_list1)
print(f"Avg CPU time = {np.mean(cput_list1):.4f} ({np.std(cput_list1):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list1):.4f} ({np.std(re_tol_list1):.4f}) seconds")

CPU times: [4.330746650695801, 3.972721576690674, 4.1175549030303955, 4.126878976821899, 4.085805416107178, 3.8205864429473877, 3.862731456756592, 2.89613938331604, 3.0432541370391846, 3.5992815494537354]
Residuals: [0.0006822281057085249, 0.001477088339888735, 0.0007804428449526912, 0.0007087772629707357, 0.0009320299909800737, 0.0009361300007083879, 0.0007263101357281746, 0.0011257631513641436, 0.0018020632566804818, 0.0008262251568905469]
Avg CPU time = 3.7856 (0.4509) seconds
Avg residual = 0.0010 (0.0004) seconds


In [13]:
fbs_gme3_cc_mat_partial = lambda D, C, sig1_C, gamma, U0, mat_x : FBS_algo.fbs_gme3_cc_mat_proto(D, C, sig1_C, gamma, U0, mat_x, admm=FBS_algo.prox_mat_l21_admm_cholsolve)
cput_list1, re_tol_list1 = run_multiple_trials_fbs_gme3_cc(
    fbs_func=fbs_gme3_cc_mat_partial,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

MPD Converged in 15 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 9 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 14 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 14 iterations.
FBS reaches max step 10.
FBS res: 0.0007804428449529367
Trial 3/10, Seed=3, CPU=3.1375, Residual=7.8044e-04
Edge (0, 8) has different weights in D1 and D2: 0.9862799434950493 vs 0.9862799434950492
Edge (3, 16) has different weights in D1 and D2: 0.97764721188625 vs 0.9776472118862499
Edge (4, 8) has different weights in D1 and D2: 0.9909056135899924 vs 0.99

MPD Converged in 6 iterations.
ADMM Converged in 43 iterations.
MPD Converged in 11 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 8 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 7 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 6 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 5 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 4 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 3 iterations.
ADMM Converged in 13 iterations.
MPD Converged in 2 iterations.
ADMM Converged in 13 iterations.
FBS reaches max step 10.
FBS res: 0.0009320299909807657
Trial 5/10, Seed=5, CPU=3.4442, Residual=9.3203e-04
Edge (6, 12) has different weights in D1 and D2: 0.9846025675509958 vs 0.9846025675509957
Edge (8, 15) has different weights in D1 and D2: 0.9955357371005883 vs 0.9955357371005882
Edge (11, 13) has different weights in D1 and D2: 0.9853502425484794 vs

Edge (1, 11) has different weights in D1 and D2: 0.9822897455867642 vs 0.9822897455867643
Edge (1, 9) has different weights in D1 and D2: 0.9775832193930524 vs 0.9775832193930523
Edge (2, 18) has different weights in D1 and D2: 0.9685218640029833 vs 0.9685218640029835
Edge (7, 13) has different weights in D1 and D2: 0.985025588808146 vs 0.9850255888081458
Edge (7, 16) has different weights in D1 and D2: 0.9778924058780931 vs 0.9778924058780932
Edge (9, 14) has different weights in D1 and D2: 0.9838853771578108 vs 0.9838853771578109
Edge (11, 15) has different weights in D1 and D2: 0.9820699314179888 vs 0.9820699314179889
Edge (12, 13) has different weights in D1 and D2: 0.9713810695277258 vs 0.9713810695277257
Edge (20, 37) has different weights in D1 and D2: 0.9618021616649084 vs 0.9618021616649083
Edge (21, 27) has different weights in D1 and D2: 0.9747986581494517 vs 0.9747986581494518
Edge (21, 25) has different weights in D1 and D2: 0.9585417444030084 vs 0.9585417444030085
Edge (2

Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edge (20, 36) has different weights in D1 and D2: 0.9785802580925324 vs 0.9785802580925322
Edge (20, 33) has different weights in D1 and D2: 0.9653963685045024 vs 0.9653963685045023
Edge (23, 27) has different weights in D1 and D2: 0.9651199900995031 vs 0.9651199900995032
Edge (25, 30) has different weights in D1 and D2: 0.9726364584496663 vs 0.9726364584496662
Edge (2

In [14]:
"""Cholesky-solve implementation """
print("CPU times:", cput_list1)
print("Residuals:", re_tol_list1)
print(f"Avg CPU time = {np.mean(cput_list1):.4f} ({np.std(cput_list1):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list1):.4f} ({np.std(re_tol_list1):.4f}) seconds")

CPU times: [2.930859327316284, 3.0367581844329834, 3.137539863586426, 3.7992708683013916, 3.4442298412323, 3.8656725883483887, 3.891638994216919, 3.0202693939208984, 3.73422908782959, 4.172112703323364]
Residuals: [0.0006822281057080558, 0.0014770883398884708, 0.0007804428449529367, 0.0007087772629716336, 0.0009320299909807657, 0.0009361300007092669, 0.0007263101357285922, 0.001125763151363996, 0.0018020632566809003, 0.0008262251568902173]
Avg CPU time = 3.5033 (0.4226) seconds
Avg residual = 0.0010 (0.0004) seconds


In [15]:
fbs_gme3_cc_mat_partial = lambda D, C, sig1_C, gamma, U0, mat_x : FBS_algo.fbs_gme3_cc_mat_proto(D, C, sig1_C, gamma, U0, mat_x, admm=FBS_algo.prox_mat_l21_admm_splusolve)
cput_list1, re_tol_list1 = run_multiple_trials_fbs_gme3_cc(
    fbs_func=fbs_gme3_cc_mat_partial,
    n_trials=n_trails,
    p=p,          # dimension
    K=K,            # clusters
    n_in_cluster=n_in_cluster, # points per cluster
    gamma=None,     # let the code pick gamma
    print_graph='n'
)

  0%|          | 0/10 [00:00<?, ?it/s]

Edge (1, 11) has different weights in D1 and D2: 0.9787099418652315 vs 0.9787099418652316
Edge (7, 9) has different weights in D1 and D2: 0.9923337052562523 vs 0.9923337052562522
Edge (8, 16) has different weights in D1 and D2: 0.9874302263841489 vs 0.987430226384149
Edge (15, 16) has different weights in D1 and D2: 0.9851365789554172 vs 0.9851365789554171
Edge (20, 29) has different weights in D1 and D2: 0.9833664343988594 vs 0.9833664343988593
Edge (22, 27) has different weights in D1 and D2: 0.9781411203131611 vs 0.978141120313161
Edge (25, 33) has different weights in D1 and D2: 0.9845795905903347 vs 0.9845795905903346
Edge (26, 38) has different weights in D1 and D2: 0.9860119844110937 vs 0.9860119844110938
Edge (27, 37) has different weights in D1 and D2: 0.9674645143549332 vs 0.9674645143549333
Edge (27, 31) has different weights in D1 and D2: 0.9685310267175351 vs 0.9685310267175352
Edge (27, 28) has different weights in D1 and D2: 0.964429941979804 vs 0.9644299419798039
Edge (

/home/chester/miniconda3/envs/ml-general/lib/python3.8/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:347: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
FBS reaches max step 10.
FBS res: 2.865362733327509e+34
Trial 1/10, Seed=1, CPU=5.9682, Residual=2.8654e+34
Edge (0, 16) has different weights in D1 and D2: 0.9495206345084938 vs 0.9495206345084937
Edge (1, 12) has different weights in D1 and D2: 0.9494840089556563 vs 0.9494840089556562
Edge (4, 6) has different weights in D1 and D2: 0.932756711851081 vs 0.9327567118510811
Edge (5, 12) has different weights in D1 and D2: 0.9655649882541968 vs 0.9655649882541969
Edge (6, 16) has different weights in D1 and D2: 0.9497861308267955 vs 0.9497861308267954
Edge (6, 17) has different weights in D1 and D2: 0.9379936765565087 vs 0.9379936765565088
Edge (7, 19) has different weights in D1 and D2: 0.9432726901749661 vs 0.9432726901749662
Edge

/home/chester/miniconda3/envs/ml-general/lib/python3.8/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:347: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


...

ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
FBS reaches max step 10.
FBS res: 5.11551177277406e+35
Trial 8/10, Seed=8, CPU=6.0708, Residual=5.1155e+35
Edge (0, 10) has different weights in D1 and D2: 0.9932506874657409 vs 0.9932506874657407
Edge (2, 14) has different weights in D1 and D2: 0.9950647333474154 vs 0.9950647333474153
Edge (4, 12) has different weights in D1 and D2: 0.9913481239486825 vs 0.9913481239486824
Edge (5, 16) has different weights in D1 and D2: 0.9913648242185197 vs 0.9913648242185198
Edge (7, 15) has different weights in D1 and D2: 0.9934886086063768 vs 0.9934886086063769
Edge (10, 15) has different weights in D1 and D2: 0.9928239968189976 vs 0.9928239968189977
Edge (11, 17) has different weights in D1 and D2: 0.9930788973006183 vs 0.9930788973006184
E

/home/chester/miniconda3/envs/ml-general/lib/python3.8/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:347: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
FBS reaches max step 10.
FBS res: 2.967427124225547e+35
Trial 9/10, Seed=9, CPU=6.6573, Residual=2.9674e+35
Edge (2, 10) has different weights in D1 and D2: 0.9792728084416201 vs 0.97927280844162
Edge (3, 9) has different weights in D1 and D2: 0.9857584065724948 vs 0.9857584065724949
Edge (3, 5) has different weights in D1 and D2: 0.9589160858573578 vs 0.9589160858573579
Edge (8, 11) has different weights in D1 and D2: 0.9818931150049255 vs 0.9818931150049256
Edge (14, 16) has different weights in D1 and D2: 0.9862284634395853 vs 0.9862284634395854
Edge (14, 17) has different weights in D1 and D2: 0.9800005400356739 vs 0.9800005400356738
Edge (16, 19) has different weights in D1 and D2: 0.9687493879831948 vs 0.9687493879831947
Edg

/home/chester/miniconda3/envs/ml-general/lib/python3.8/site-packages/scipy/sparse/linalg/_dsolve/linsolve.py:347: SparseEfficiencyWarning: splu converted its input to CSC format
  warn('splu converted its input to CSC format', SparseEfficiencyWarning)


ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
ADMM reaches max step 50.
FBS reaches max step 10.
FBS res: 2.1153163220857313e+33
Trial 10/10, Seed=10, CPU=6.5591, Residual=2.1153e+33


In [16]:
"""Splu-solve implementation """
print("CPU times:", cput_list1)
print("Residuals:", re_tol_list1)
print(f"Avg CPU time = {np.mean(cput_list1):.4f} ({np.std(cput_list1):.4f}) seconds")
print(f"Avg residual = {np.mean(re_tol_list1):.4f} ({np.std(re_tol_list1):.4f}) seconds")

CPU times: [5.968165397644043, 6.58363151550293, 4.29287576675415, 5.3538596630096436, 6.35126256942749, 5.128771781921387, 5.900534629821777, 6.070809364318848, 6.657347679138184, 6.559056997299194]
Residuals: [2.865362733327509e+34, 2.8737576731276946e+31, 7.883421564062768e+35, 2.038788656202728e+34, 2.0793734553751695e+32, 5.752849458052474e+33, 1.0351907091298304e+31, 5.11551177277406e+35, 2.967427124225547e+35, 2.1153163220857313e+33]
Avg CPU time = 5.8866 (0.7202) seconds
Avg residual = 165379275261103819821341447133593600.0000 (264385223366778536364505827494592512.0000) seconds
